# Performance Analysis: Batch Valuation

This notebook benchmarks the `get_straddle_backtests()` function against the traditional per-straddle `get_straddle_valuation()` loop.

## Goals
1. Benchmark each phase independently
2. Compare batch vs loop performance
3. Validate output correctness
4. Document performance characteristics

In [ ]:
import time
import sys
from pathlib import Path
from collections import defaultdict

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from specparser.amt import (
    get_straddle_backtests,
    get_straddle_valuation,
    find_straddle_yrs,
    load_all_prices,
    set_prices_dict,
    clear_prices_dict,
    xpry, xprm,
)

print("Imports successful!")

## Configuration

In [ ]:
# Paths
AMT_PATH = "../data/amt.yml"
PRICES_PATH = "../data/prices.parquet"

# Test parameters (small scale)
PATTERN_SMALL = "^LA Comdty"  # Single asset for focused testing
START_YEAR = 2022
END_YEAR = 2024

# Test parameters (large scale)
PATTERN_LARGE = "Comdty$"  # All commodities

print(f"Small pattern: {PATTERN_SMALL}")
print(f"Large pattern: {PATTERN_LARGE}")
print(f"Year range: {START_YEAR}-{END_YEAR}")

## Phase 1: Load Prices

In [ ]:
# Load prices once
clear_prices_dict()

start = time.perf_counter()
prices_dict = load_all_prices(
    PRICES_PATH,
    f"{START_YEAR-1}-01-01",
    f"{END_YEAR}-12-31"
)
set_prices_dict(prices_dict)
load_time = time.perf_counter() - start

print(f"Loaded {len(prices_dict):,} price entries in {load_time:.2f}s")

## Phase 2: Warm Up Caches

In [ ]:
# Warm up YAML loader cache
_ = find_straddle_yrs(AMT_PATH, START_YEAR, END_YEAR, PATTERN_SMALL, True)

# Warm up Numba JIT
print("Warming up Numba JIT...")
_ = get_straddle_backtests(PATTERN_SMALL, START_YEAR, START_YEAR, AMT_PATH)
print("Done!")

## Phase 3: Small Scale Benchmark (Single Asset)

In [ ]:
# Get straddle list
straddles_table = find_straddle_yrs(AMT_PATH, START_YEAR, END_YEAR, PATTERN_SMALL, True)
print(f"Found {len(straddles_table['rows'])} straddles")

# Build task list for loop approach
straddle_counts = defaultdict(int)
for row in straddles_table['rows']:
    asset, straddle = row[0], row[1]
    year = xpry(straddle)
    month = xprm(straddle)
    straddle_counts[(asset, year, month)] += 1

tasks = []
for (asset, year, month), count in sorted(straddle_counts.items()):
    for i in range(count):
        tasks.append((asset, year, month, i))

print(f"Tasks: {len(tasks)}")

In [ ]:
# Benchmark batch approach
iterations = 5

start = time.perf_counter()
for _ in range(iterations):
    batch_result = get_straddle_backtests(PATTERN_SMALL, START_YEAR, END_YEAR, AMT_PATH, valid_only=True)
batch_time = (time.perf_counter() - start) / iterations * 1000

print(f"Batch (valid_only=True): {len(batch_result['rows']):,} rows in {batch_time:.1f}ms")

In [ ]:
# Benchmark loop approach
start = time.perf_counter()
loop_rows = []
loop_columns = None
for asset, year, month, i in tasks:
    try:
        result = get_straddle_valuation(asset, year, month, i, AMT_PATH)
        if loop_columns is None and result['rows']:
            loop_columns = result['columns']
        mv_idx = result['columns'].index('mv')
        loop_rows.extend([r for r in result['rows'] if r[mv_idx] != '-'])
    except Exception:
        pass
loop_time = (time.perf_counter() - start) * 1000

print(f"Loop (filtered): {len(loop_rows):,} rows in {loop_time:.1f}ms")
print(f"Rate: {loop_time/len(tasks):.2f}ms/straddle")

In [ ]:
# Small scale comparison
speedup_small = loop_time / batch_time

print("=" * 60)
print("SMALL SCALE COMPARISON")
print("=" * 60)
print(f"Pattern: {PATTERN_SMALL}")
print(f"Year range: {START_YEAR}-{END_YEAR}")
print(f"Straddles: {len(tasks)}")
print("-" * 60)
print(f"Loop approach:  {loop_time:8.1f}ms ({loop_time/len(tasks):.2f}ms/straddle)")
print(f"Batch approach: {batch_time:8.1f}ms")
print("-" * 60)
print(f"Speedup: {speedup_small:.1f}x")
print("=" * 60)

## Phase 4: Large Scale Benchmark (All Commodities)

In [ ]:
# Get straddle list for large scale
straddles_large = find_straddle_yrs(AMT_PATH, START_YEAR, END_YEAR, PATTERN_LARGE, True)
print(f"Found {len(straddles_large['rows'])} straddles for {PATTERN_LARGE}")

# Build task list
straddle_counts = defaultdict(int)
for row in straddles_large['rows']:
    asset, straddle = row[0], row[1]
    year = xpry(straddle)
    month = xprm(straddle)
    straddle_counts[(asset, year, month)] += 1

tasks_large = []
for (asset, year, month), count in sorted(straddle_counts.items()):
    for i in range(count):
        tasks_large.append((asset, year, month, i))

print(f"Tasks: {len(tasks_large)}")

In [ ]:
# Benchmark batch approach (large scale)
iterations = 3

start = time.perf_counter()
for _ in range(iterations):
    batch_large = get_straddle_backtests(PATTERN_LARGE, START_YEAR, END_YEAR, AMT_PATH, valid_only=True)
batch_time_large = (time.perf_counter() - start) / iterations * 1000

print(f"Batch (large): {len(batch_large['rows']):,} rows in {batch_time_large:.1f}ms")

In [ ]:
# Benchmark loop approach (large scale)
start = time.perf_counter()
loop_rows_large = []
loop_columns = None
for asset, year, month, i in tasks_large:
    try:
        result = get_straddle_valuation(asset, year, month, i, AMT_PATH)
        if loop_columns is None and result['rows']:
            loop_columns = result['columns']
        mv_idx = result['columns'].index('mv')
        loop_rows_large.extend([r for r in result['rows'] if r[mv_idx] != '-'])
    except Exception:
        pass
loop_time_large = (time.perf_counter() - start) * 1000

print(f"Loop (large): {len(loop_rows_large):,} rows in {loop_time_large:.1f}ms")
print(f"Rate: {loop_time_large/len(tasks_large):.2f}ms/straddle")

In [ ]:
# Large scale comparison
speedup_large = loop_time_large / batch_time_large

print("=" * 60)
print("LARGE SCALE COMPARISON")
print("=" * 60)
print(f"Pattern: {PATTERN_LARGE}")
print(f"Year range: {START_YEAR}-{END_YEAR}")
print(f"Straddles: {len(tasks_large)}")
print("-" * 60)
print(f"Loop approach:  {loop_time_large:8.1f}ms ({loop_time_large/len(tasks_large):.2f}ms/straddle)")
print(f"Batch approach: {batch_time_large:8.1f}ms")
print("-" * 60)
print(f"Speedup: {speedup_large:.1f}x")
print("=" * 60)

## Phase 5: Output Validation

In [ ]:
# Compare row counts
print("Output Row Comparison:")
print(f"  Batch rows: {len(batch_result['rows']):,}")
print(f"  Loop rows:  {len(loop_rows):,}")
print(f"\nBatch columns: {batch_result['columns']}")
print(f"Loop columns:  {loop_columns}")

In [ ]:
# Sample output from batch
print("Sample batch output (first 5 rows):")
for row in batch_result['rows'][:5]:
    print(f"  {row[0]:12s} {row[2]} mv={row[10][:12]}")

In [ ]:
# Sample output from loop
print("Sample loop output (first 5 rows):")
mv_idx = loop_columns.index('mv')
date_idx = loop_columns.index('date')
asset_idx = loop_columns.index('asset')
for row in loop_rows[:5]:
    print(f"  {row[asset_idx]:12s} {row[date_idx]} mv={row[mv_idx][:12]}")

## Performance Summary

In [ ]:
print("=" * 70)
print("PERFORMANCE SUMMARY")
print("=" * 70)
print()
print("Small scale (LA Comdty, 2022-2024):")
print(f"  - Straddles: {len(tasks)}")
print(f"  - Loop:  {loop_time:.0f}ms ({loop_time/len(tasks):.2f}ms/straddle)")
print(f"  - Batch: {batch_time:.0f}ms (valid_only=True)")
print(f"  - Speedup: {speedup_small:.1f}x")
print()
print("Large scale (Comdty$, 2022-2024):")
print(f"  - Straddles: {len(tasks_large)}")
print(f"  - Loop:  {loop_time_large:.0f}ms ({loop_time_large/len(tasks_large):.2f}ms/straddle)")
print(f"  - Batch: {batch_time_large:.0f}ms (valid_only=True)")
print(f"  - Speedup: {speedup_large:.1f}x")
print()
print("Notes:")
print("  - Loop benefits from heavy memoization (schedules, tickers, prices)")
print("  - Batch uses Numba-accelerated model computation")
print("  - Batch bottleneck: Python-based price lookup (~68% of time)")
print("  - For further speedup, price lookup needs Cython/Numba optimization")
print("=" * 70)